In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import re


In [4]:
data = pd.read_csv('/mnt/c/Users/AdamC/Plocha/Skola/bachelors_thesis/data/wiki/train.csv')

In [5]:
data["non_toxic"] = data.iloc[:,2:8].apply(lambda x: 1 if (sum(x)==0) else 0, axis=1)
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,class_binary
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,1
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,1
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,1
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,1
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,1


In [6]:
cleaned_data = data.copy()

# Removing Hyperlinks from text
cleaned_data["comment_text"] = cleaned_data["comment_text"].map(lambda x: re.sub(r"https?://\S+|www\.\S+", "", x))
# Rmoving emojis from text
cleaned_data["comment_text"] = cleaned_data["comment_text"].map(lambda x: re.sub("["
                                                                                 u"\U0001F600-\U0001F64F"
                                                                                 u"\U0001F300-\U0001F5FF"
                                                                                 u"\U0001F680-\U0001F6FF"
                                                                                 u"\U0001F1E0-\U0001F1FF"
                                                                                 u"\U00002702-\U000027B0"
                                                                                 u"\U000024C2-\U0001F251"
                                                                                 "]+", "", x, flags=re.UNICODE))
# Removing IP addresses from text
cleaned_data["comment_text"] = cleaned_data["comment_text"].map(
    lambda x: re.sub(r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", "", x))

In [7]:
# Removing html tags from text
cleaned_data["comment_text"] = cleaned_data["comment_text"].map(lambda x: re.sub(r"<.*?>", "", x))
cleaned_data["comment_text"] = cleaned_data["comment_text"].map(
    lambda x: re.sub(r"\"\"", "\"", x))  # replacing "" with "
cleaned_data["comment_text"] = cleaned_data["comment_text"].map(
    lambda x: re.sub(r"^\"", "", x))  # removing quotation from start and the end of the string
cleaned_data["comment_text"] = cleaned_data["comment_text"].map(lambda x: re.sub(r"\"$", "", x))
# Removing Punctuation / Special characters (;:'".?@!%&*+) which appears more than twice in the text
cleaned_data["comment_text"] = cleaned_data["comment_text"].map(
    lambda x: re.sub(r"[^a-zA-Z0-9\s][^a-zA-Z0-9\s]+", " ", x))
# Removing Special characters
cleaned_data["comment_text"] = cleaned_data["comment_text"].map(lambda x: re.sub(r"[^a-zA-Z0-9\s\"\',:;?!.()]", " ", x))
# Removing extra spaces in text
cleaned_data["comment_text"] = cleaned_data["comment_text"].map(lambda x: re.sub(r"\s\s+", " ", x))
cleaned_data.reset_index(inplace=True)
cleaned_data.drop(columns=["index"], inplace=True)


cleaned_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,class_binary
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,1
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,1
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,1
3,0001b41b1c6bb37e,\nMore\nI can't make any real suggestions on i...,0,0,0,0,0,0,1
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,1


In [8]:
cleaned_data.to_csv(
    '/mnt/c/Users/AdamC/Plocha/Skola/bachelors_thesis/data/wiki/preprocessed/train.csv')

In [10]:
cleaned_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,class_binary
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,1
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,1
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,1
3,0001b41b1c6bb37e,\nMore\nI can't make any real suggestions on i...,0,0,0,0,0,0,1
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,1


In [12]:
data = cleaned_data.copy()
C_indexs = data[data["non_toxic"]==1].index

toxic_count = data[data["non_toxic"]==0].shape[0]
drop_indxs = np.random.choice(C_indexs, size=data[data["non_toxic"]==1].shape[0] - toxic_count, replace=False)

# Randomly selecting and dropping non_toxic comments from datasets and leaving 5000 behind
data.drop(drop_indxs, inplace=True)

data.to_csv("/mnt/c/Users/AdamC/Plocha/Skola/bachelors_thesis/data/wiki/preprocessed/train_50_50.csv")
print(data.shape)

(32450, 9)
